# Batch computation of UMAP coordinates for individual trials
Tim Tyree<br>
6.24.2021

In [1]:
# #automating the boring stuff may take ~20 seconds to initialize here...
from lib.getterdone import *

__What this notebook does__
- give an easy way to loop through to get mass outputs
- get the raw xy umap coordinates for individual trials that satisfy a customizable modality/condition

__Schema for UMAP results__
- phase - integer label of observation (0 for dubious, 1 for decisive, and 2 for reflective)
- t - time of observation (seconds) s.t. t=0 corresponds to stimulus start time
- x - the x coordinate of umap results
- y - the y coordinate of umap results

# simple interface for selecting which trials/data to consider

## select a folder that contains input .csv files

In [2]:
use_search_gui=False
warnings=True
printing=True

print("Please search for a file from within the folder you wish to compute UMAP results for (see pop-up window)...")
if use_search_gui:
    file=search_for_file()
else:
    #previously selected files
    file="/Users/timothytyree/Documents/GitHub/neurophysics/notebooks/Data/update-6-5-2021/Archie_SRT_Set211_Subset1_200518_130717_trialData.csv"

file_lst=get_all_files_matching_pattern(file,'.csv')
data_folder=os.path.dirname(file)
os.chdir(data_folder)
#compute the set of all modnames in data_folder
modname_lst=sorted(set([parse_for_modname(file) for file in file_lst]))
print(f"There were {len(modname_lst)} trial sets found in data_folder: {data_folder}" )

Please search for a file from within the folder you wish to compute UMAP results for (see pop-up window)...
There were 55 trial sets found in data_folder: /Users/timothytyree/Documents/GitHub/neurophysics/notebooks/Data/update-6-5-2021


## (optional) import a token trial set from that folder

In [3]:
#load a single set of trials into notebook memory
j=2
modname=modname_lst[j]
t_min_considered, number_of_neurons, dict_spike_times, dict_trial_times, dict_trial_data = load_dataset(modname,data_folder,
                                                                                                        warnings=warnings,
                                                                                                       printing=printing)
boo_all_files_found=determine_whether_all_files_were_found(t_min_considered, number_of_neurons, dict_spike_times, dict_trial_times, dict_trial_data)
if boo_all_files_found:
    if printing:
        print(f'All files were found for trial set: {modname}.')
    setnum=parse_set_number(modname)
else:
    if printing:
        print(f'Warning: not all files were found for trial set: {modname}. Consider a different j')

All files were found for trial set: Archie_SRT_Set212_Subset1_200520_165716_.


In [4]:
#load data for one trial and view an example pandas.Dataframe from 1 set
df = pd.DataFrame(dict_trial_data).T
df['PicID_1'] = [s.lower() for s in df['PicID_1'].values]
df.head()

,imName,imNum,imPheeName,imMatchFlag,novel,imOff,vpltTrial,Block,PheeID,PicID_1,PicID_2,PidID_3
1,JennyRightGood3.JPG,2,none,0,1,3.5081116752699,1,1,none,jenny,Right,2
2,ChewieRightGood1.JPG,3,none,0,1,3.50763143645599,1,1,none,chewie,Right,3
3,HermesFrontGood2.JPG,4,none,0,1,3.50854450464249,1,1,none,hermes,Front,4
4,HankRightGood1.jpg,5,hank,1,1,2.66679767635651,1,1,hank,hank,Right,5
5,HermesFrontGood2.JPG,4,none,0,0,3.50858635362238,1,1,none,hermes,Front,4


In [5]:
#get querying options that are availble in the current trial set 
# PicID_1_lst=sorted(set(df.PicID_1.values))
PheeID_lst=sorted(set(df.PheeID.values))
orientation_lst=sorted(set(df.PicID_2.values))
imname_lst=sorted(set(df.imName.values))

# #print which options were found
# print(orientation_lst)
# print(sorted(set(df.imPheeName.values)))
# print(imname_lst)

## define a function that selects which trials to consider

For example,

```imMatchFlag=1
vpltTrial=1  # as stimulus was presented
output_folder=f'selecting_imMatchFlag_{imMatchFlag}'
def my_query_function(df):
    boo =(df.vpltTrial==vpltTrial)
    boo&=df.imMatchFlag==imMatchFlag
    return boo```

Here, you may define a custom my_query_function that takes a pandas.Dataframe of your trial set as input and returns a boolean index, `boo` that is `True` for any trial that you want to compute UMAP results for.

In [6]:
#################################
# Examples of my_query_function
#################################
imMatchFlag=1
vpltTrial=1  # as stimulus was presented
output_folder=f'selecting_imMatchFlag_{imMatchFlag}'
def my_query_function(df):
    boo =(df.vpltTrial==vpltTrial)
    boo&=df.imMatchFlag==imMatchFlag
    return boo

# output_folder='selecting_alltrials'
# def my_query_function(df):
#     '''selects every trial'''
#     return True

In [7]:
#perform a query to identify the trials to be considered from this trial set
boo=my_query_function(df)
trialnum_include_values=df[boo].index.values

block_lst=sorted(set(df[boo].Block.values))
setnum=parse_set_number(modname)
if printing:
    print(f"output_folder: {output_folder}")
    print(f"trials matching query for trial set {setnum} only: {sorted(trialnum_include_values)}")
    print(f"blocks matching query for trial set {setnum} only: {block_lst}")

output_folder: selecting_imMatchFlag_1
trials matching query for trial set 212 only: [4, 8, 12, 16, 18, 23, 25, 31, 36, 38, 45, 68, 69, 70, 71, 72, 73, 82, 84, 106, 113, 121, 127, 131, 140, 143, 145, 149, 166, 170, 174, 175, 181, 187, 190, 195, 199, 203, 204, 207, 208, 237, 238, 239, 240, 241, 246, 252, 260, 263, 264, 265, 288, 292, 294, 298, 302, 304, 306, 309, 313, 319, 342, 346, 348, 352, 354, 377, 379, 381, 382, 384, 386, 388, 389, 391, 392, 397, 398, 412, 417, 432, 443, 444]
blocks matching query for trial set 212 only: [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 26]


## (optional) additional examples for my_query_function

In [8]:
# output_folder='selecting_copper'
# def my_query_function(df):
#     '''Query function that selects a given monkey, where a stimulus is presented.'''
#     #choose constant values
#     vpltTrial=1  # as stimulus was presented
#     # monkname=PheeID_lst[4]
#     # monkname=PheeID_lst[3]#cooper
#     monkname='copper'
#     # orientation='Front'#'none'
#     #select trials of relevance
#     boo=df.vpltTrial==vpltTrial
#     boo&=df.imPheeName==monkname #may be redundant
#     boo&=df.PheeID==monkname     #may be redundant
#     # boo&=df.orientation==orientation 
#     return boo

# imMatchFlag=3
# output_folder=f'selecting_imMatchFlag_{imMatchFlag}'
# def my_query_function(df):
#     vpltTrial=1  # as stimulus was presented
#     monkname_lst=[
# #         'aladdin',
# #          'ares',
# #          'chewie',
# #          'copper',
# #          'dip',
# #          'han',
# #          'hank',
# #          'hermes',
# #          'jasmine',
#          'mowgli',
#          'none',
# #          'poseidon',
# #          'raja',
# #          'waylon'
#     ]
#     boo =False&(df.vpltTrial==1)
#     for monkname in monkname_lst:
#         boo|=df.imPheeName==monkname #may be redundant
#         boo|=df.PheeID==monkname     #may be redundant
#     boo&=(df.vpltTrial==1)
#     boo&=df.imMatchFlag==imMatchFlag
#     return boo

# compute and save umap xy coordinates for individual trials

In [9]:
def get_routine(output_folder,data_folder):
    def routine(modname):
        '''
        routine computes umap xy coordinates for each trial in the trial set indicated by modname located in data_folder.
        trials are only considered if selected by my_query_function.
        saves umap xy coordinates for each trial as a .csv file in the folder, output_folder.
        returns a list of lists of output .csv directories
        '''
        ################################
        # Modify Routine Parameters Here
        ################################
        #parameters you might be interested in changing
        t_arrival=1.5 #seconds
        t_departure=2.5 #seconds
        t_arrival_physical=.15 #seconds
        t_stop=3. #seconds
        umap_kwargs={
            'metric'      :'cosine',#'correlation',
            'random_state':42, # for random_state fixed, the umap results are repeatable to machine precision 
        #     'min_dist'    :0.5,
        #     'spread'      :2
        }
        output_metric='hyperbolic'
        # output_metric='gaussian'
        
        #parameters I don't think you'll be interested in changing
        use_cache0=True
        nid_self=None #automatically select the most spiking neuron to be nid_self
        # nid_self=12 #set nid_self to the 13th neuron. #nid_self decides time points to take observations as spike times of some neuron
        warnings=False
        printing=False
        min_num_obs=1 #the minimum number of spikes nid_self needs to have for this trial to be considered
        cache_folder=data_folder+'/cache/'
        
        ################################
        # Main Routine
        ################################
        os.chdir(data_folder)
        #load a single set of trials into notebook memory
        t_min_considered, number_of_neurons, dict_spike_times, dict_trial_times, dict_trial_data = load_dataset(modname,data_folder,warnings=warnings,printing=printing)
        boo_all_files_found=determine_whether_all_files_were_found(t_min_considered, number_of_neurons, dict_spike_times, dict_trial_times, dict_trial_data)
        if not boo_all_files_found:
            if printing:
                print(f'Warning: not all files were found for trial set: {modname}.')
            return None

        #determine which trial to consider using my_query_function
        df = pd.DataFrame(dict_trial_data).T
        df['PicID_1'] = [s.lower() for s in df['PicID_1'].values]
        boo=my_query_function(df)
        trialnum_include_values=df[boo].index.values

        
        output_dir_lst=[]
        # loop over all trials
        for trialnum in trialnum_include_values:
            trial_kwargs=dict(df.loc[trialnum])
            output_modname=return_output_modname(modname,trialnum,trial_kwargs,
                                                 make_output_self_documenting=True)
            output_fn=output_modname+f'_nid_self_{nid_self}.csv'
            output_dir=os.path.join(output_folder,output_fn)

            #identify the neuron with the most spikes as nid_self
            nid_self=0
            max_obs=0
            for nid in range(number_of_neurons):
                obs=dict_spike_times[nid].shape[0]
                if obs>max_obs:
                    max_obs=obs
                    nid_self=nid
            
            if nid_self is not None:
                save_fn=modname+f'compressedPointProcess_nid_self_{nid_self}.npz'
            else:
                save_fn='none'
                
            os.chdir(data_folder)
            os.chdir(cache_folder)
            #compute the point process and cache in save_fn
            if (not os.path.exists(save_fn)) or (not use_cache0):
                #compute the point process model
                nid_self,target,data,trialnum_values,t_values=compute_point_process_data(t_min_considered, number_of_neurons, 
                                                                                         dict_spike_times, dict_trial_times,
                                                                                         dict_trial_data,
                                                                                         printing=printing,
                                                                                         nid_self=nid_self,
                                                                                         t_arrival=t_arrival,
                                                                                         t_departure=t_departure,
                                                                                         t_arrival_physical=t_arrival_physical,
                                                                                         t_stop=t_stop)

                #change directories to cache location
                os.chdir(data_folder)
                os.chdir(cache_folder)
                #cache the resulting point process as a compressed numpy array for later reuse
                save_fn=modname+f'pointProcess_{nid_self}_compressed.npz'
                np.savez_compressed(save_fn, target=target,data=data,trialnum_values=trialnum_values,t_values=t_values)
                if printing:
                    print(f'For nid_self={nid_self}, the resulting pointprocess was saved in: \n\t{save_fn}')

            #load the point process from cache
            cached_fn=os.path.abspath(save_fn)
            dataset=np.load(save_fn)
            #extract lag feature vector
            data=dataset['data']
            target=dataset['target']
            trialnum_values=dataset['trialnum_values']
            spike_times=dataset['t_values']
            target_names=[r'dubious',r'decisive',r'reflective']

            #query trials by trial
            # match=True
            # mismatch=False
            # novel=False
            query=trialnum_values==trialnum
            dat=data[query,:]
            targ=target[query]
            t_values_out=t_values[query]
            num_obs=dat.shape[0]

            if num_obs>=min_num_obs:
                # if printing:
                #     print(f"fitting umap to the {num_obs} observations considered in this trial...")
                #TODO: compute umap xy coordinates, 
                if output_metric == 'hyperbolic':
                    x_values,y_values = get_hyperbolic_umap_xycoordinates(dat,**umap_kwargs) #computes the underlying class instance and returns the xy values for each observation
                elif output_metric == 'gaussian':
                    x_values,y_values = get_gaussian_umap_xycoordinates(dat,**umap_kwargs) #computes the underlying class instance and returns the xy values for each observation

                # save umap coordinates to .csv
                successfully_saved_bool=save_umap_xycoords_to_csv(output_dir,x_values,y_values,targ,t_values_out)
                output_dir_lst.append (output_dir)

        return output_dir_lst
    return routine

In [10]:
#define function for the main routine
npartitions=os.cpu_count()
output_folder=os.path.abspath(output_folder)
print(f'number of cores found: {npartitions}')

#initialize output_folder if it doesn't already exist
os.chdir(data_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
os.chdir(output_folder)
#initialize cache_folder if it doesn't already exist
os.chdir(data_folder)
cache_folder=data_folder+'/cache/'
if not os.path.exists(cache_folder):
    os.mkdir(cache_folder)

routine=get_routine(output_folder,data_folder)


number of cores found: 8


In [11]:
#test importing of trial set data. stdout should read True
t_min_considered, number_of_neurons, dict_spike_times, dict_trial_times, dict_trial_data = load_dataset(modname,data_folder,warnings=warnings,printing=printing)
boo_all_files_found=determine_whether_all_files_were_found(t_min_considered, number_of_neurons, dict_spike_times, dict_trial_times, dict_trial_data)
boo_all_files_found

True

In [ ]:
# #test the routine. stdout should be a list of file names
# routine(modname)

In [ ]:
# #test for smaller runtimes because of autocaching caching is nonzero routine. stdout should be a list of file names
# routine(modname)

In [ ]:
#for each trial set...
bag = db.from_sequence(modname_lst, npartitions=npartitions).map(routine)
print(f"saving xy coordinates for the trials satistfying my_query_function.\nlooking in {len(retval_lst)} trial sets.\noutputing to .csv in output_folder={output_folder}...")
start = time.time()
retval_lst = list(bag)
print(f"the run time for computing umap individual xy coordinates was {(time.time()-start)/60:.2f} minutes.")

In [ ]:
beep(1)

In [ ]:
#look at all the xy coordinates you just made!
os.chdir(output_folder)
!ls

In [ ]:
retval_lst

# plot UMAP xy coordinates for a given trial in a given trial set

In [ ]:
modnum=2
trialnum=4
modname=modname_lst[modnum]

# compute a umap fit for the point process sampled from a trial given by trialnum
t_min_considered, number_of_neurons, dict_spike_times, dict_trial_times, dict_trial_data = load_dataset(modname,data_folder,
                                                                                                        warnings=warnings,
                                                                                                       printing=printing)
#determine which trial to consider using my_query_function
df = pd.DataFrame(dict_trial_data).T
df['PicID_1'] = [s.lower() for s in df['PicID_1'].values]
boo=my_query_function(df)
trialnum_include_values=df[boo].index.values
nid_self=None

nid_self,target,data,trialnum_values,t_values=compute_point_process_data(t_min_considered, number_of_neurons, 
                                                                                     dict_spike_times, dict_trial_times,
                                                                                     dict_trial_data,
                                                                                     printing=printing,
                                                                                     nid_self=nid_self,
                                                                                     t_arrival=t_arrival,
                                                                                     t_departure=t_departure,
                                                                                     t_arrival_physical=t_arrival_physical,
                                                                                     t_stop=t_stop)
query=trialnum_values==trialnum

#query by computing the boolean index for a given condition
dat=data[query,:]
targ=target[query]
t_values_out=t_values[query]
num_obs=dat.shape[0]
#define color values
c_values=targ

umap_kwargs={
    'metric'      :'cosine',#'correlation',
    'random_state':42, # for random_state fixed, the umap results are repeatable to machine precision 
#     'min_dist'    :0.5,
#     'spread'      :2
}

print(f"fitting umap to the {num_obs} observations considered in this trial...")
output_metric='hyperbolic'
if output_metric == 'hyperbolic':
    # gaussian_mapper=fit_gaussian_mapper(data,**kwargs) #returns the underlying class instance
    x_values,y_values = get_hyperbolic_umap_xycoordinates(dat,**umap_kwargs) #computes the underlying class instance and returns the xy values for each observation

print(f"plotting 2D projections...")   
print(f"...these plots can be saved using the function, fig.savefig.")   
fig=PlotUmapCoordsHyperbolic(x_values,y_values,c_values)
fig.show()

In [ ]:
# saves umap coordinates to .csv
trial_kwargs=dict(df.loc[trialnum])
output_modname=return_output_modname(modname,trialnum,trial_kwargs,
                                     make_output_self_documenting=True)
output_fn=output_modname+'.csv'
output_dir=os.path.join(data_folder,output_folder,output_fn)

output_dir=os.path.join(data_folder,output_folder,output_fn)
successfully_saved_bool=save_umap_xycoords_to_csv(output_dir,x_values,y_values,targ,t_values_out)

In [ ]:
if use_search_gui:
    input_dir=search_for_file()
else:
    input_dir=output_dir
#load a given umap xycoordinate csv file for the hyperbolic embedding
x_values,y_values,targ,t_values_out=load_umap_xycoords_from_csv(input_dir)
fig=PlotUmapCoordsHyperbolic(x_values,y_values,c_values)
fig.show()

In [ ]:
# compute a umap fit for the point process sampled from each trial
output_metric='gaussian'
if output_metric == 'gaussian':
    # gaussian_mapper=fit_gaussian_mapper(data,**kwargs) #returns the underlying class instance
    x_values,y_values = get_gaussian_umap_xycoordinates(dat,**umap_kwargs) #computes the underlying class instance and returns the xy values for each observation
           
fig=PlotUmapCoordsGaussian(x_values,y_values,c_values)
fig.show()

In [ ]:
print('all jobs completed successfully!')
beep(3)